In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import networkx as nx
import plotly.graph_objects as go
import ast

pd.set_option('display.max_columns', 50)

# Introdução

Em nossas análises, temos como meta principal o mapeamento dos jogos com maior taxa de engajamento


Nesse sentido, após analisar as variáveis, decidimos por eliminar algumas delas:

'playerage' - Nos parece ter alguns problemas de formatação (problemas com a tabela ascii) e entrega um faixa de idade recomendada pelos jogadores. Consideramos muito subjetivo e prefirimos usar a 'minage', apesar de serem valores próximos na média;

'gamelink' - Se quisermos ver o jogo no BGG, basta pesquisá-lo por ser nome;

'boardgameexpansion' - Cada jogo tem a sua específica, não faz sentido analisar;

'max_community' (Fornecido pela comunidade) - No geral, igual ao 'maxplayers' (fornecido pelos fabricantes), só que com alguns valores faltando; Em geral, dados fornecidos pelo público podem conter problemas, como falta de votos em certo jogo, mas no geral são mais confiáveis.

'minplayers' - Fornecido pelos fabricantes, indicam o limite de jogabilidade, mas não garante uma experiência interessante. Sendo assim, vale usar a opinião da comunidade;

In [5]:
df = pd.read_excel('df_unificado.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'df_unificado.xlsx'

In [ ]:
df.head()

,object_id,name,yearpublished,sortindex,minplayers,maxplayers,minplaytime,maxplaytime,minage,min_community,...,boardgameexpansion_cnt,boardgameversion_cnt,boardgamefamily_cnt,boardgamedesigner,boardgameartist,boardgamepublisher,boardgamehonor,boardgamecategory,boardgamemechanic,boardgamefamily
0,174430,Gloomhaven,2017,1,1,4,60,120,12,3.0,...,4,19,7,"['Isaac Childres""']","['Alexandr Elichev', 'Josh T. McDowell', 'Alva...","['Cephalofair Games', 'Albi', 'Asmodee', 'Feue...",['2017 Best Science Fiction or Fantasy Board G...,"['Adventure', 'Exploration', 'Fantasy', 'Fight...","['Campaign / Battle Card Driven', 'Cooperative...","['Campaign Games', 'Components: Miniatures', '..."
1,161936,Pandemic Legacy Season 1,2015,2,2,4,60,60,13,4.0,...,0,33,3,"['Rob Daviau""', 'Matt Leacock""']",['Chris Quilliams'],"['Z-Man Games', 'Asterion Press', 'Devir', 'Fi...",['2015 Cardboard Republic Immersionist Laurel ...,"['Environmental', 'Medical']","['Action Points', 'Cooperative Game', 'Hand Ma...","['Campaign Games', 'Legacy', 'Pandemic']"
2,167791,Terraforming Mars,2016,3,1,5,120,120,12,3.0,...,15,29,6,"['Jacob Fryxelius""']",['Isaac Fryxelius'],"['FryxGames', 'Arclight', 'Fantasmagoria', 'Gh...",['2016 Cardboard Republic Architect Laurel Nom...,"['Economic', 'Environmental', 'Industry / Manu...","['Card Drafting', 'End Game Bonuses', 'Hand Ma...","['Fryxgames Future Timeline', 'Planets: Mars',..."
3,182028,Through the Ages A New Story of Civilization,2015,4,2,4,120,120,14,3.0,...,1,14,2,"['Vlaada Chv\\u00e1til""']","['Filip Murmak', 'Radim Pech', 'Jakub Politzer...","['Czech Games Edition', 'Cranio Creations', 'D...",['2015 Golden Geek Best Strategy Board Game No...,"['Card Game', 'Civilization', 'Economic']","['Action Points', 'Auction/Bidding', 'Auction:...","['Tableau Building', 'Through the Ages']"
4,224517,Brass Birmingham,2018,5,2,4,60,120,14,3.0,...,0,9,6,"['Gavan Brown""', 'Matt Tolman""', 'Martin Walla...","['Lina Cossette', 'David Forest', 'Damien Mamm...","['Roxley', 'BoardM Factory', 'Conclave Editora...",['2018 Golden Geek Best Board Game Artwork & P...,"['Economic', 'Industry / Manufacturing', 'Tran...","['Hand Management', 'Income', 'Loans', 'Market...","['Beer', 'Brass', 'Cities: Birmingham (England..."


Após removermos as variáveis que eram explicitamente descartáveis, devemos remover as que contém dados aparentemente ruins e arrumar as que têm má formatação

In [ ]:
# 'min_community' possui alguns valores nulos
min_df = df[['minplayers', 'min_community']]
min_df.dropna()
min_df.describe()

,minplayers,min_community
count,20016.000000,14149.000000
mean,2.055256,3.203336
std,0.745471,1.398786
min,0.000000,1.000000
25%,2.000000,2.000000
50%,2.000000,3.000000
75%,2.000000,4.000000
max,10.000000,31.000000


Vou preencher os valores nulos de 'min_community' com 'minplayers' + 2. Prefirimo os 'min_community' pela maior confiabilidade das opiniões dos avaliadores do BGG.

In [ ]:
#decidimos por subistituir os valores nulos de 'min_community' por 'minplayers' + 2
df['min_community'].fillna(df['minplayers']+2, inplace=True)
df = df.drop("minplayers", axis=1)

In [ ]:
#'usersrated' e 'totalvotes' são muito similares gráficamente, então usaremos a razão média entre eles para preencher os nulos
#provavelmente usaremos ambos em cluster no futuro
df['totalvotes'].fillna(df['usersrated']*df['totalvotes'].mean()/df['usersrated'].mean(), inplace=True)
df['usersrated'].fillna(df['totalvotes']*df['usersrated'].mean()/df['totalvotes'].mean(), inplace=True)
df[['totalvotes', 'usersrated']].describe()

,totalvotes,usersrated
count,20016.000000,20016.000000
mean,13.453664,739.363636
std,57.153155,3095.427307
min,0.000000,0.000000
25%,1.000000,47.000000
50%,3.000000,105.000000
75%,7.000000,333.250000
max,1868.000000,90730.000000


In [ ]:
#'average' e 'baverage' também são muito parecidos e aparentam possuir uma razão média constante. Trataremos da mesma forma que anteriormente com 'usersrated' e 'totalvotes'
df['average'].fillna(df['baverage']*df['average'].mean()/df['baverage'].mean(), inplace=True)
df['baverage'].fillna(df['average']*df['baverage'].mean()/df['average'].mean(), inplace=True)
#agora vamos remover os que são nulos em ambos
df = df[df['average'].isnull() == False]
df[['average', 'baverage']].describe()

,average,baverage
count,20014.000000,20014.000000
mean,6.276287,5.296803
std,1.066911,1.493120
min,0.000000,0.000000
25%,5.715585,5.502662
50%,6.357130,5.544965
75%,6.955065,5.675688
max,9.442860,8.584240


Validação

In [ ]:
df.query('objectid == 35677')['baverage']

NameError: name 'df' is not defined

A partir daqui, o nosso DF não possui mais nenhum coluna com valores nulos ou desformatados

In [ ]:
#vamos separar nossos dados para podermos progredir melhor
df_popularidade = df[['totalvotes','usersrated','numgeeklists','numtrading','numwanting','numcomments','siteviews','numplays','numplays_month','news','blogs','weblink','podcast']]
df_qualidade = df[['average','baverage','stddev','boardgamehonor_cnt']]
#as outras colunas são todas descrições dos jogos, que usaremos depois